In [45]:
import os
import json
import pandas as pd
from datetime import datetime, timedelta
from collections import defaultdict

directory = os.fsencode('data/wtt_matches')

In [46]:
players = []
genders = defaultdict(lambda: None)

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if not filename.endswith(".json"):
        continue

    path = os.path.join(f'data/wtt_matches', filename) 
    with open(path, 'r') as f:
        matches = json.load(f)

    for m in matches:
        gender = m['documentCode'][3]

        isTeam = False
        for c in m['competitiors']:
            if isTeam:
                break

            for p in c['players']:
                p['gender'] = gender
                pid = int(p['playerId'])
                if pid > 100000000 or pid < 100000:
                    isTeam = True
                if isTeam:
                    break

                if not p['playerOrgCode']:
                    p['playerOrgCode'] = c['competitiorOrg']
                players.append(p)

        if isTeam and m['teamParentData']:
            for _m in m['teamParentData']['extended_info']['matches']:
                if not _m.get('match_result'):
                    continue

                mm = _m['match_result']
                for c in mm['competitiors']:
                    for p in c['players']:
                        p['gender'] = gender
                        pid = int(p['playerId'])
                        if pid > 100000000 or pid < 100000:
                            continue

                        if not p['playerOrgCode']:
                            p['playerOrgCode'] = c['competitiorOrg']
                        players.append(p)


In [47]:
pf = pd.DataFrame(players)
pf.drop(columns=['playerPosition'], inplace=True)
pf

,playerId,playerName,playerGivenName,playerFamilyName,playerOrgCode,gender
0,115910,LIN Gaoyuan,None,None,CHN,M
1,137237,LIN Shidong,None,None,CHN,M
2,133848,YUAN Licen,None,None,CHN,M
3,135888,XIANG Peng,None,None,CHN,M
4,121404,FAN Zhendong,None,None,CHN,M
...,...,...,...,...,...,...
44299,111050,Alvaro ROBLES,Alvaro,ROBLES,ESP,M
44300,112221,Lily ZHANG,Lily,ZHANG,USA,W
44301,102445,Offiong EDEM,Offiong,EDEM,NGR,W
44302,111833,Suthasini SAWETTABUT,Suthasini,SAWETTABUT,THA,W


In [48]:
pf[pf.playerName == 'Lubomir PISTEJ']

,playerId,playerName,playerGivenName,playerFamilyName,playerOrgCode,gender
9135,107445,Lubomir PISTEJ,Lubomir,PISTEJ,SVK,M
9261,107445,Lubomir PISTEJ,Lubomir,PISTEJ,SVK,M
39146,107445,Lubomir PISTEJ,Lubomir,PISTEJ,SVK,M
39249,107445,Lubomir PISTEJ,Lubomir,PISTEJ,SVK,X
39303,107445,Lubomir PISTEJ,Lubomir,PISTEJ,SVK,M
39345,107445,Lubomir PISTEJ,Lubomir,PISTEJ,SVK,M


In [49]:
pf[pf.playerId.isna()]
# only 2, manually fix

,playerId,playerName,playerGivenName,playerFamilyName,playerOrgCode,gender


In [50]:
name_conflicts = pf.groupby('playerId')['playerName'].apply(set)
conf = pd.DataFrame(name_conflicts)
conf['cnt'] = name_conflicts.apply(len)
conf[conf.cnt > 1]


,playerName,cnt
playerId,,
100032,"{Farah ABDELAZIZ, ABDEL-AZIZ Farah}",2
100439,"{SALEH Ahmed, Ahmed SALEH}",2
100486,"{Gaston ALTO, ALTO Gaston}",2
100621,"{Tiago APOLONIA, APOLONIA Tiago}",2
100696,"{ASSAR Omar, Omar ASSAR}",2
...,...,...
200310,"{ADEGOKE Muizz Olawale, ADEGOKE Muizz}",2
200361,"{RIOS ROMERO Carlos Josue Alberto, RIOS Carlos}",2
200391,"{MENEZES Abimael, MENEZES ARAUJO Abimael}",2


In [51]:
id_conflicts = pf.groupby('playerName')['playerId'].apply(set)
idconf = pd.DataFrame(id_conflicts)
idconf['cnt'] = id_conflicts.apply(len)
idconf[idconf.cnt > 1]
# These actually might be different people... 

,playerId,cnt
playerName,,
LEE Daeun,"{132702, 135391}",2
Yang WANG,"{112735, 109995}",2


In [52]:
org_conflicts = pf.groupby('playerId')['playerOrgCode'].apply(set)
oconf = pd.DataFrame(org_conflicts)
oconf['cnt'] = org_conflicts.apply(len)
oconf[oconf.cnt > 1]


,playerOrgCode,cnt
playerId,,
100868,"{SVK, POL/SVK, SVK/CZE, CZE/SVK, SVK/FRA, SVK/...",7
102380,"{ENG, GBR, ENG/GER}",3
103163,"{USA, USA/SUI}",2
103903,"{ROU/ESP, ROU}",2
104359,"{SVK/SRB, SRB}",2
...,...,...
137627,"{AUS/SUI, FRA/AUS, AUS, ESP/AUS, NOR/AUS, SVK/...",7
146048,"{PER, PER/ECU}",2
200322,"{KAZ, LUX/KAZ}",2


In [53]:
cleaned = []
for id, rows in pf.groupby('playerId'):
    clrow = {
        'id': id,
        'org': None,
        'name': None,
        'gender': 'X',
    }
    for row in rows.itertuples():
        if row.gender != 'X':
            clrow['gender'] = row.gender
        if not clrow['name']:
            clrow['name'] = row.playerName
        if not clrow['org']:
            clrow['org'] = row.playerOrgCode
        elif row.playerOrgCode:
            # take shorter country code, don't want doubles codes
            if len(row.playerOrgCode) < len(clrow['org']):
                clrow['org'] = row.playerOrgCode

    cleaned.append(clrow)
cf = pd.DataFrame(cleaned)

In [54]:
pf[pf.playerId.isin(cf[cf.org.isna()].id)]

,playerId,playerName,playerGivenName,playerFamilyName,playerOrgCode,gender


In [55]:
cf.to_csv('data/wtt_cleaned/players.tsv', index=False, sep='\t')